What is Phidata?
Phidata is a framework for building multi-modal agents and workflows.

Build agents with **memory**, **knowledge**, **tools** and **reasoning**.
Build teams of agents that can work together to solve problems.
Interact with your agents and workflows using a beautiful Agent UI.

Agents use tools to take actions and interact with external systems.

Tools are functions that an Agent can run to achieve tasks. For example: searching the web, running SQL, sending an email or calling APIs. You can use any python function as a tool or use a pre-built toolkit

In [61]:
import os
from dotenv import load_dotenv
from phi.agent import Agent,RunResponse
from phi.model.groq import Groq
from phi.tools.duckduckgo import DuckDuckGo
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

##### 1. Web Search Agent

In [5]:
web_search_agent = Agent(
    name="Web Search Agent",
    model=Groq(id="llama3-groq-70b-8192-tool-use-preview"),
    tools=[DuckDuckGo()],
    instructions=["Always include sources"],
    show_tool_calls=True,
    markdown=True,)

In [12]:
# web_search_agent.print_response("whats happenning in India", stream=True,markdown=True)
# web_search_agent.print_response("Share a 2 sentence horror story.")
response = web_search_agent.run("Whats happening in Indian politics ?")

In [17]:
print(response.content)


Running:
 - duckduckgo_news(query=Indian politics, max_results=5)

Here are the latest updates on Indian politics:
1. **PM to launch development projects in Delhi** - The PM will also launch several development projects, worth Rs 12,200 crore, in the poll-bound national capital. [Read more](https://indianexpress.com/article/political-pulse/today-in-politics-first-bjp-candidates-list-pm-modi-rally-delhi-9760529/)

2. **US-India technology cooperation** - National security adviser Jake Sullivan will travel to India for a visit focusing on cooperation on strategic technology, including in defense, space and artificial intelligence. [Read more](https://www.msn.com/en-us/news/world/the-last-high-profile-trip-to-india-by-the-biden-administration-will-focus-on-technology/ar-AA1wVF9f)

3. **Ram Charan's movie event** - Ram Charan is gearing up for the release of his upcoming movie Game Changer, co-starring Kiara Advani. On Saturday, the makers held an event in Andhra Pradesh where the actor's

##### 2. Financial Agent

In [19]:
from phi.tools.yfinance import YFinanceTools

In [28]:
finance_agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    name="Finance Agent",
    tools=[YFinanceTools(stock_price=True,stock_fundamentals=True,company_news=True,historical_prices=True)],
    deescription="You are an investment analyst that researches stocks prices, commany analysis and market trends. Recommends stocks to your clients.",
    instructions=["Format your response using markdown and use tables to disply stock analysis where possible."],
    show_tool_calls=True,
    markdown=True
)

In [29]:
finance_agent.print_response(message="Give me 2 stocks that I can buy for long term.",markdown=True,stream=True)

Output()

##### 3. Multi-agent orchestration

Phidata agents support text, images, audio and video.

For example, you can create an image agent that can understand images and make tool calls as needed

In [31]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.yfinance import YFinanceTools

In [39]:
web_agent = Agent(
    name="web Agent",
    role="Search the web for latest information, not older than 2 months",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools = [DuckDuckGo()],
    instructions=["Always search for financial websites for the latest news."],
    description="You are stock market analyst, who searches the latest company information, \
    its projects, milestones, financials, reports, future vision, company officials.",
    show_tool_calls=True,
    markdown=True
)

In [40]:
finance_agent = Agent(
    name='finance_agent',
    role="Get the financial data and analyses it.",
    model = Groq(id="llama-3.3-70b-versatile"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions=["Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

In [43]:
agent_team = Agent(
    provider=Groq(),
    team=[web_agent, finance_agent],
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

In [46]:
agent_team.print_response("Summarize analyst recommendations and share the latest news for NVDA",stream=True)

Output()

##### 4. Writing custom tool

For more control, write your own python functions and add them as tools to an Agent.

In [147]:
import json
import httpx

from phi.agent import Agent
from phi.model.groq import Groq

In [146]:
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to return. Defaults to 10.

    Returns:
        str: JSON string of top stories.
    """

    # Fetch top story IDs
    response = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json')
    story_ids = response.json()

    # Fetch story details
    stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json')
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        stories.append(story)
    return json.dumps(stories)

agent = Agent(tools=[get_top_hackernews_stories], model=Groq(),show_tool_calls=True, markdown=True)
agent.print_response("Summarize the top 5 stories on hackernews?", stream=True)


Output()